In [1]:
#import pandas and read in the two CSV files
import pandas as pd

file_1 = "raw_data/schools_complete.csv"
file_2 = "raw_data/students_complete.csv"

schools_df = pd.read_csv(file_1)
students_df = pd.read_csv(file_2)

In [2]:
#Check out schools file
schools_df.sort_values(by = ['size'])

,School ID,name,type,size,budget
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858
4,4,Griffin High School,Charter,1468,917500
14,14,Thomas High School,Charter,1635,1043130
2,2,Shelton High School,Charter,1761,1056600
10,10,Wright High School,Charter,1800,1049400
6,6,Cabrera High School,Charter,1858,1081356
5,5,Wilson High School,Charter,2283,1319574
13,13,Ford High School,District,2739,1763916
0,0,Huang High School,District,2917,1910635


In [3]:
#check out students file
students_df.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


# District Summary

In [4]:
#Extract and store all the desired quantities
num_schools = len(schools_df)
num_students = len(students_df)
total_budget = schools_df['budget'].sum()
avg_math_score = students_df["math_score"].sum()/num_students
avg_reading_score = students_df["reading_score"].sum()/num_students
perc_pass_math = 100*len(students_df.loc[students_df["math_score"] >= 70, :])/num_students
perc_pass_read = 100*len(students_df.loc[students_df["reading_score"] >= 70, :])/num_students
pass_rate = (perc_pass_math + perc_pass_read)/2


In [5]:
district_summary = pd.DataFrame([{"Number of Schools": num_schools,
               "Total Students": num_students,
               "Total Budget": total_budget,
              "Average Math Score": avg_math_score,
              "Average Reading Score": avg_reading_score,
              "% Passing Math": perc_pass_math,
              "% Passing Reading": perc_pass_read,
              "% Overall Pass Rate": pass_rate}])
district_summary = district_summary[ ["Number of Schools",
                                      "Total Students",
                                      "Total Budget",
                                      "Average Math Score",
                                      "Average Reading Score",
                                      "% Passing Math",
                                      "% Passing Reading",
                                      "% Overall Pass Rate"] ]
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)

district_summary


,Number of Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Pass Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.393158


# School Summary

In [6]:
schools_in_merge = schools_df.copy()
schools_in_merge["per_student_budget"] = schools_in_merge["budget"]/schools_in_merge["size"]

students_in_merge = students_df.copy()


"""
Create two columns with 1 if passing math/reading; 0 if not. Summing gives number passing; divide by number of
students to get percent.
"""
#Could do this with binning
students_in_merge["pass_reading"] = students_in_merge["reading_score"].map( lambda x: 1 if x > 70 else 0 )
students_in_merge["pass_math"] = students_in_merge["math_score"].map( lambda x: 1 if x > 70 else 0 )
students_in_merge = students_in_merge.groupby(["school"]).sum()
students_in_merge = students_in_merge[ ['reading_score','math_score','pass_reading','pass_math'] ]
students_in_merge = students_in_merge.reset_index()
students_in_merge = students_in_merge.rename(columns = {'school':'name'})

"""
Merge tables and normalize columns to get averages and percentages
"""
school_summary = pd.merge(schools_in_merge, students_in_merge, on = 'name')
school_summary['math_score'] = school_summary['math_score' ]/school_summary['size']
school_summary['reading_score'] = school_summary['reading_score' ]/school_summary['size']
school_summary['pass_reading'] = 100 * school_summary['pass_reading' ]/school_summary['size']
school_summary['pass_math'] = 100 * school_summary['pass_math' ]/school_summary['size']
school_summary['pass_rate'] = (school_summary['pass_math'] + school_summary['pass_reading'])*0.5

school_summary = school_summary.set_index("name")
school_summary.drop(['School ID'], axis = 1, inplace = True)

school_summary = school_summary.rename( columns = {"type" : "School Type",
                                 "size": "Total Students",
                                 "budget": "Total School Budget",
                                 "per_student_budget": "Per Student Budget",
                                 "reading_score":"Average Reading Score",
                                 "math_score":"Average Math Score",
                                 "pass_reading":"% Passing Reading",
                                 "pass_math": "% Passing Math",
                                 "pass_rate":"% Overall Passing Rate"} )

school_summary["Total School Budget"] = school_summary["Total School Budget"].map("${:,.2f}".format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${:,.2f}".format)
school_summary["Total Students"] = school_summary["Total Students"].map("{:,}".format)


school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
name,,,,,,,,,
Huang High School,District,"2,917","$1,910,635.00",$655.00,81.182722,76.629414,78.813850,63.318478,71.066164
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,81.158020,76.711767,78.433367,63.750424,71.091896
Shelton High School,Charter,"1,761","$1,056,600.00",$600.00,83.725724,83.359455,92.617831,89.892107,91.254969
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,80.934412,77.289752,78.187702,64.746494,71.467098
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.816757,83.351499,93.392371,89.713896,91.553134
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.989488,83.274201,93.254490,90.932983,92.093736
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.975780,83.061895,93.864370,89.558665,91.711518
Bailey High School,District,"4,976","$3,124,928.00",$628.00,81.033963,77.048432,79.300643,64.630225,71.965434
Holden High School,Charter,427,"$248,087.00",$581.00,83.814988,83.803279,92.740047,90.632319,91.686183


# Top Performing Schools

In [7]:
top_schools = school_summary.sort_values(by = ["% Overall Passing Rate"], ascending = False).head(5)
top_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
name,,,,,,,,,
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.989488,83.274201,93.254490,90.932983,92.093736
Pena High School,Charter,962,"$585,858.00",$609.00,84.044699,83.839917,92.203742,91.683992,91.943867
Wright High School,Charter,"1,800","$1,049,400.00",$583.00,83.955000,83.682222,93.444444,90.277778,91.861111
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.975780,83.061895,93.864370,89.558665,91.711518
Holden High School,Charter,427,"$248,087.00",$581.00,83.814988,83.803279,92.740047,90.632319,91.686183


# Bottom Performing Schools

In [8]:
bottom_schools = school_summary.sort_values(by = ["% Overall Passing Rate"], ascending = True).head(5)
bottom_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,80.744686,76.842711,77.744436,64.066017,70.905226
Huang High School,District,"2,917","$1,910,635.00",$655.00,81.182722,76.629414,78.813850,63.318478,71.066164
Johnson High School,District,"4,761","$3,094,650.00",$650.00,80.966394,77.072464,78.281874,63.852132,71.067003
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,81.158020,76.711767,78.433367,63.750424,71.091896
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,80.934412,77.289752,78.187702,64.746494,71.467098


# Math Scores By Grade

In [9]:
math_students = students_df.copy()
math_students['number_of_students'] = 1
math_grades = math_students.groupby(["school", "grade"]).sum()
math_grades = math_grades[ ['number_of_students','math_score'] ]
math_grades = math_grades.reset_index()
math_grades['math_score'] = math_grades['math_score']/math_grades['number_of_students']
math_grades = math_grades.pivot(index = 'school', columns = 'grade', values = 'math_score')
math_grades

grade,10th,11th,12th,9th
school,,,,
Bailey High School,76.996772,77.515588,76.492218,77.083676
Cabrera High School,83.154506,82.765560,83.277487,83.094697
Figueroa High School,76.539974,76.884344,77.151369,76.403037
Ford High School,77.672316,76.918058,76.179963,77.361345
Griffin High School,84.229064,83.842105,83.356164,82.044010
Hernandez High School,77.337408,77.136029,77.186567,77.438495
Holden High School,83.429825,85.000000,82.855422,83.787402
Huang High School,75.908735,76.446602,77.225641,77.027251
Johnson High School,76.691117,77.491653,76.863248,77.187857


# Reading Scores By Grade

In [10]:
reading_students = students_df.copy()
reading_students['number_of_students'] = 1
reading_grades = reading_students.groupby(["school", "grade"]).sum()
reading_grades = reading_grades[ ['number_of_students','reading_score'] ]
reading_grades = reading_grades.reset_index()
reading_grades['reading_score'] = reading_grades['reading_score']/reading_grades['number_of_students']
reading_grades.pivot(index = 'school', columns = 'grade', values = 'reading_score')

grade,10th,11th,12th,9th
school,,,,
Bailey High School,80.907183,80.945643,80.912451,81.303155
Cabrera High School,84.253219,83.788382,84.287958,83.676136
Figueroa High School,81.408912,80.640339,81.384863,81.198598
Ford High School,81.262712,80.403642,80.662338,80.632653
Griffin High School,83.706897,84.288089,84.013699,83.369193
Hernandez High School,80.660147,81.396140,80.857143,80.866860
Holden High School,83.324561,83.815534,84.698795,83.677165
Huang High School,81.512386,81.417476,80.305983,81.290284
Johnson High School,80.773431,80.616027,81.227564,81.260714


# Scores by School Spending

In [11]:
student_group = students_df.copy()
student_group["pass_reading"] = student_group["reading_score"].map( lambda x: 1 if x > 70 else 0 )
student_group["pass_math"] = student_group["math_score"].map( lambda x: 1 if x > 70 else 0 )
student_group['number_of_students'] = 1
student_group = student_group.groupby(['school']).sum()
student_group[ ['reading_score', 'math_score', 'pass_reading', 'pass_math', 'number_of_students'] ]
student_group = student_group.reset_index()
student_group = student_group.rename(columns = {'school':'name'})

schools_spend = schools_df.copy()
schools_spend["per_student_budget"] = schools_spend["budget"]/schools_spend["size"]

spend_summary = pd.merge(student_group, schools_spend, on = "name")
spend_summary = spend_summary[ [ 'name','reading_score',
                                'math_score','pass_reading',
                                'pass_math','number_of_students','per_student_budget' ] ]

bins = [0, 600, 628, 644, 660]
bin_label = ["<600", ' 600 - 628', '628 - 644', '644 - 660']
spend_summary['Spending Range($)'] =  pd.cut(spend_summary['per_student_budget'], bins, labels = bin_label )
spend_summary = spend_summary[ ['reading_score',
                                'math_score',
                                'pass_reading',
                                'pass_math',
                                'number_of_students',
                               'per_student_budget',
                               'Spending Range($)'] ]


spend_summary = spend_summary.groupby(['Spending Range($)']).sum()
spend_summary['reading_score'] = spend_summary['reading_score']/spend_summary['number_of_students']
spend_summary['math_score'] = spend_summary['math_score']/spend_summary['number_of_students']
spend_summary['pass_reading'] = 100 * spend_summary['pass_reading']/spend_summary['number_of_students']
spend_summary['pass_math'] = 100 * spend_summary['pass_math']/spend_summary['number_of_students']
spend_summary['pass_rate'] = 0.5 * (spend_summary['pass_reading'] + spend_summary['pass_math'] )
spend_summary = spend_summary[['math_score', 'reading_score', 'pass_math', 'pass_reading', 'pass_rate']]

spend_summary = spend_summary.rename(columns = {"math_score": "Average Math Score",
                                               "reading_score": "Average Reading Score",
                                               "pass_math": "% Passing Math",
                                               "pass_reading":"% Passing Reading",
                                               "pass_rate":"% Overall Pass Rate"} )


spend_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Pass Rate
Spending Range($),,,,,
<600,83.362283,83.912412,90.232501,93.271005,91.751753
600 - 628,79.179989,81.976641,73.116392,83.769916,78.443154
628 - 644,77.821056,81.301007,68.168168,80.056527,74.112348
644 - 660,77.049297,81.005604,64.062373,78.372452,71.217412


# Scores by School Size

In [12]:
student_group = students_df.copy()
student_group["pass_reading"] = student_group["reading_score"].map( lambda x: 1 if x > 70 else 0 )
student_group["pass_math"] = student_group["math_score"].map( lambda x: 1 if x > 70 else 0 )
student_group['number_of_students'] = 1
student_group = student_group.groupby(['school']).sum()
student_group[ ['reading_score', 'math_score', 'pass_reading', 'pass_math', 'number_of_students'] ]
student_group = student_group.reset_index()
student_group = student_group.rename(columns = {'school':'name'})

schools_spend = schools_df.copy()
schools_spend["per_student_budget"] = schools_spend["budget"]/schools_spend["size"]

size_summary = pd.merge(student_group, schools_spend, on = "name")
size_summary = size_summary[ [ 'name','reading_score',
                                'math_score','pass_reading',
                                'pass_math','number_of_students','per_student_budget', 'size' ] ]


bins = [0, 1800, 3000, 5000]
bin_label = ['Small(<1800)', 'Medium(1800 - 3000)', 'Large(3000 - 5000)']
size_summary['School Size'] =  pd.cut(size_summary['size'], bins, labels = bin_label )
size_summary = size_summary[ ['reading_score',
                                'math_score',
                                'pass_reading',
                                'pass_math',
                                'number_of_students',
                               'School Size'] ]

size_summary = size_summary.groupby(['School Size']).sum()
size_summary['reading_score'] = size_summary['reading_score']/size_summary['number_of_students']
size_summary['math_score'] = size_summary['math_score']/size_summary['number_of_students']
size_summary['pass_reading'] = 100 * size_summary['pass_reading']/size_summary['number_of_students']
size_summary['pass_math'] = 100 * size_summary['pass_math']/size_summary['number_of_students']
size_summary['pass_rate'] = 0.5 * (size_summary['pass_reading'] + size_summary['pass_math'] )
size_summary = size_summary[['math_score', 'reading_score', 'pass_math', 'pass_reading', 'pass_rate']]

size_summary = size_summary.rename(columns = {"math_score": "Average Math Score",
                                             "reading_score": "Average Reading Score",
                                              "pass_math":"% Passing Math",
                                             "pass_reading":"% Passing Reading",
                                             "pass_rate":"% Overall Pass Rate"})

size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Pass Rate
School Size,,,,,
Small(<1800),83.523035,83.861418,90.264498,92.959146,91.611822
Medium(1800 - 3000),78.878001,81.993096,72.713008,83.226110,77.969559
Large(3000 - 5000),77.070764,80.928365,64.335093,78.417070,71.376082


# Scores By School Type

In [13]:
student_group = students_df.copy()
student_group["pass_reading"] = student_group["reading_score"].map( lambda x: 1 if x > 70 else 0 )
student_group["pass_math"] = student_group["math_score"].map( lambda x: 1 if x > 70 else 0 )
student_group['number_of_students'] = 1
student_group = student_group.groupby(['school']).sum()
student_group[ ['reading_score', 'math_score', 'pass_reading', 'pass_math', 'number_of_students'] ]
student_group = student_group.reset_index()
student_group = student_group.rename(columns = {'school':'name'})

schools_spend = schools_df.copy()
schools_spend["per_student_budget"] = schools_spend["budget"]/schools_spend["size"]

type_summary = pd.merge(student_group, schools_spend, on = "name")
type_summary = type_summary[ [ 'name','reading_score',
                                'math_score','pass_reading',
                                'pass_math','type', 'size' ] ]

type_summary = type_summary.groupby(['type']).sum()
type_summary['reading_score'] = type_summary['reading_score']/type_summary['size']
type_summary['math_score'] = type_summary['math_score']/type_summary['size']
type_summary['pass_reading'] = 100*type_summary['pass_reading']/type_summary['size']
type_summary['pass_math'] = 100*type_summary['pass_math']/type_summary['size']
type_summary['pass_rate'] = 0.5*(type_summary['pass_math'] + type_summary['pass_reading'])

type_summary = type_summary.rename(columns = {"reading_score": "Average Reading Score",
                                             "math_score": "Average Math Score",
                                             "pass_reading":"% Passing Reading",
                                             "pass_math":"% Passing Math",
                                             "size":"Number of Students",
                                             "pass_rate":"% Overall Pass Rate"} )



type_summary

,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Number of Students,% Overall Pass Rate
type,,,,,,
Charter,83.902821,83.406183,93.152370,90.282106,12194,91.717238
District,80.962485,76.987026,78.369662,64.305308,26976,71.337485
